In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import numpy as np
import dask
import xarray as xr
import xmitgcm
from xmitgcm import open_mdsdataset
from xhistogram.xarray import histogram

from matplotlib import cm
import matplotlib.colors as colors
import numpy as np

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

from bbl import *
from osse import *
from canyon_utils import *
from sloped_MITgcm import *
from wmt import *

In [48]:
# Constant parameters

Γ = 0.0008613659531090722
N = np.sqrt(g*α*Γ)
θ = 1.26E-3
f = -5.3e-5

h = 230
k0 = 5.2e-5
k1 = 1.8e-3

dx = 600.; dy = 600.
delta_t = 15.

### Loading output and pre-processing

In [49]:
experiments = ["idealized-plane-1D", "idealized-plane-1D-Pr230", "idealized-plane", "idealized-canyon", "idealized-sill", "canyon-tracer"]
data = {}

for exp in experiments:
    data_dir = f"../../MITgcm/experiments/rotated_BBTRE_{exp}/run/"
    if "1D" in exp:
        Δt = 2*delta_t
    else:
        Δt = delta_t
    ds = open_mdsdataset(data_dir, ignore_unknown_vars=True, prefix=["budgetDiag"], delta_t=Δt)
    if exp=='canyon-tracer':
        ds=ds.sel(time=ds['time'][:200:10])
    else:
        ds=ds.sel(time=ds['time'])
    ds = add_thalweg(ds)
    ds, grid = add_rotated_coords(ds, θ)
    
    # For standing eddy component
    timemean = open_mdsdataset(data_dir, ignore_unknown_vars=True, prefix=["standardDiag"], delta_t=Δt).mean("time")
    timemean, timemean_grid = add_rotated_coords(timemean, θ)
    
    ds["ADVr_TH_standing"] = (timemean["WVEL"]*timemean_grid.interp(timemean["THETA"], "Z", boundary="fill").chunk({"Zl":-1})*timemean["rA"]).drop("Depth")
    ds = add_temp_budget(ds, grid, Γ, θ, add_standing=True)
    
    # Stratification and masks
    ds['Nsq'] = g*α*(Γ +
            (grid.diff(ds['THETA'].where(ds['hFacC']!=0.), 'Z', boundary='fill')) /
            grid.interp(-ds['drF'], 'Z', boundary='extend')
    ).chunk({'Zl':-1})
    mask_sill = ((ds.H - ds['Depth']) < 500)
    ds['Nsq_trough'] = ds['Nsq'].where( (irregular_section(ds['Nsq'], ds['j_thalweg'].values, width=5)) ).where(mask_sill)
    ds['Nsq_outside'] = irregular_section(ds['Nsq'], ds['j_thalweg'].values, width=21, invert_mask=True)
    ds['Nsq_flanks'] = ds['Nsq'].where((np.isnan(ds['Nsq_trough'])) & (np.isnan(ds['Nsq_outside'])))

    ds['Depth_trough'] = ds['Depth'].where( (irregular_section(ds['Depth'], ds['j_thalweg'].values, width=5)) ).where(mask_sill)
    ds['Depth_outside'] = irregular_section(ds['Depth'], ds['j_thalweg'].values, width=21, invert_mask=True)
    ds['Depth_flanks'] = ds['Depth'].where((np.isnan(ds['Depth_trough'])) & (np.isnan(ds['Depth_outside'])))
    
    ds = ds.reset_coords(['Depth'])
    ds['time'] = ds['time'].astype('float64')*1e-9/86400.
    print("Budget closes: ", check_temp_budget_closes(ds), f" (over {ds['time'].size} timesteps).", end='\n')
    
    data[exp] = {'ds': ds, 'grid': grid}

/home/hdrake/.conda/envs/bblturb-analysis/lib/python3.9/site-packages/pandas/core/indexes/base.py:4307: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
/home/hdrake/.conda/envs/bblturb-analysis/lib/python3.9/site-packages/xarray/core/indexes.py:117: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  new_index = index[indexer]
/home/hdrake/.conda/envs/bblturb-analysis/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


Budget closes:  True  (over 20 timesteps).


/home/hdrake/.conda/envs/bblturb-analysis/lib/python3.9/site-packages/pandas/core/indexes/base.py:4307: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
/home/hdrake/.conda/envs/bblturb-analysis/lib/python3.9/site-packages/xarray/core/indexes.py:117: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  new_index = index[indexer]
/home/hdrake/.conda/envs/bblturb-analysis/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


Budget closes:  True  (over 20 timesteps).


/home/hdrake/.conda/envs/bblturb-analysis/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


...

Budget closes:  True  (over 20 timesteps).


/home/hdrake/.conda/envs/bblturb-analysis/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


Budget closes:  True  (over 20 timesteps).


/home/hdrake/.conda/envs/bblturb-analysis/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


Budget closes:  True  (over 20 timesteps).


### Computing derived variables

In [50]:
budget_vars = [
    'total_tendency_TH_truth', 'total_tendency_TH',
    'diff_tendency_TH', 'adv_tendency_TH',
    'conv_adv_flux_anom_TH', 'conv_vert_adv_flux_TH', "conv_vert_adv_standing_flux_TH"
]

for exp in experiments:
    ds = data[exp]['ds']
    ds_mean = ds[budget_vars].mean(dim='time').sum(dim=['XC','YC'], skipna=True).compute()
    
    Lx, Ly = ds.XC.size*dx, ds.YC.size*dy
    ds_mean["a"] = ((ds["dV"]/ds["drF"]).sum(["XC", "YC"])/(Lx/Ly)).compute()
    
    Ly_ref = 60.e3
    cint = k0*Γ*np.cos(θ)*Ly*Lx
    cbot = k1*Γ*np.cos(θ)*Ly*Lx
    
    # Coarsened budget terms for plots
    for var in budget_vars:
        ds_mean[var+"_K-per-year"] = (ds_mean[var]/(ds_mean['drF']*Lx*Ly)*(86400.*365.)) #.coarsen(Z=5).mean().rename({'Z': 'Z_coarse'})
        ds_mean[var+"_cumint_mSv"] = (np.cumsum(ds_mean[var][::-1])[::-1]/(Γ*np.sin(θ)*Lx*Ly)*(Ly_ref*1e-3))

    # Stratification 
    ds = add_hab_coordinates(ds, grid, vert_coord='Zl', gridface='C')
    ds_mean = ds_mean.assign_coords({'Zl_habC': ds['Zl_habC']})
    ds_mean['Nsq'] = hab_interp(ds['Nsq'], vert_coord='Zl').mean(['time', 'XC', 'YC'], skipna=True).compute()
    for region in ['trough', 'flanks', 'outside']:
        ds_mean[f'Nsq_{region}'] = (
            hab_interp(ds[f'Nsq_{region}'], vert_coord='Zl').mean(['time', 'XC', 'YC'], skipna=True).compute()
        )
        
    # Misc. variables used for plotting
    ds_mean['Depth'] = ds['Depth']
    crest_HAMS = irregular_section(ds['Depth'], ds['j_thalweg'].values, width=35, invert_mask=True)
    ds_mean['crest_HAMS'] = ds['Depth'].max().values - (crest_HAMS.mean(['XC', 'YC'], skipna=True).values)
    ds_mean['YC_thalweg'] = ds['YC_thalweg']
    
    # Fill output dictionary
    data[exp]['ds_mean'] = ds_mean

/home/hdrake/.conda/envs/bblturb-analysis/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/hdrake/.conda/envs/bblturb-analysis/lib/python3.9/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/nobackup1b/users/hdrake/bblturb/notebooks/BBTRE_physics_analysis/sloped_MITgcm.py:64: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  da_itp = xr.apply_ufunc(
/home/hdrake/.conda/envs/bblturb-analysis/lib/python3.9/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/nobackup1b/users/hdrake/bblturb/notebooks/BBTRE_physics_analysis/sloped_MITgcm.py:64: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs

## Saving calculations

#### Save budget terms

In [51]:
for exp in experiments:
    saving = data[exp]['ds_mean'].copy()
    saving.to_netcdf(f"../../data/BBTRE-physics/buoyancy_{exp}.nc")